In [1]:
# !pip install ray
# !pip install "ray[default]"
# !pip uninstall grpcio
#!pip install lxml
import requests
from time import time 
from bs4 import BeautifulSoup
from tqdm import tqdm
import ray
basic_url = "https://www.10000recipe.com"
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
ingridient_seperator = "                                                                                    "

In [3]:
# ray for loop 안에서 잦은 get 지양

In [4]:
# lxml 사용시 htmlParseEntityRef: expecting ';' warning 뜸
# 실행은 되지만 warning 문을 출력해서 log에 뜸
# 그래서 html.parser 사용

In [2]:
@ray.remote
def crawling_recipe_path_list(page):
    recipe_path_list = []
    url = f"{basic_url}/recipe/list.html?order=date&page={page}"
    response = requests.get(url, headers = headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        print(f"{page} 페이지 접속 불가")
        return []
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        for a in soup.find("ul","common_sp_list_ul").find_all("a","common_sp_link"):
            recipe_path_list.append(a.get("href"))
    except:
        print(f"{page}페이지 에러 뜸 / 빈 리스트 반환")
        return []
        
    return recipe_path_list

In [3]:
# try catch로 오류 처리 해주기
def parsing_recipe_ingredients(soup):
    ingredient_list = []
    for ul in (soup.find("div", "ready_ingre3").find_all("ul")):
        for li in ul.find_all("li"):
            ingredient = {}
            ingredient['name'] = (li.text.split("\n")[0].split(ingridient_seperator)[0].strip(" "))
            ingredient['quantity'] = (li.find("span","ingre_unit").text.strip(" "))
            ingredient_list.append(ingredient)
    return ingredient_list

In [4]:
def parsing_recipe_sequences(soup):
    sequence_list = []
    for idx, div in enumerate(soup.find("div","view_step").find_all("div","view_step_cont")):
        sequence = {}
        sequence['sequence'] = idx+1
        sequence['content'] = div.text
        sequence_list.append(sequence)
    return sequence_list

In [12]:
@ray.remote
def makeing_recipe(recipe_path):
    url = basic_url+recipe_path
    response = requests.get(url, headers=headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        pass
    soup = BeautifulSoup(response.text, "html.parser")
    
    recipe = {}
    recipe['recipe_id'] = recipe_path.split("/")[2]
    recipe['img'] = soup.find("img", "portrait").get("src")
    try:
        recipe['title'] = soup.find("div","view2_summary st3").find("h3").text
    except:
        print(f"이름 오류 {recipe_path}")
        
    try:
        recipe['size'] = soup.find("span","view2_summary_info1").text
    except:
        print(f"몇인분 오류 {recipe_path}")
        
    try:
        recipe['ingredients'] = parsing_recipe_ingredients(soup)
    except:
        print(f"재료 오류 {recipe_path}")
        
    try:
        recipe['sequences'] = parsing_recipe_sequences(soup)
    except:
        print(f"조리법 오류 {recipe_path}")
    
    return recipe

In [6]:
ray.init()

2022-07-31 20:48:58,420	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-31_20-48-56_712102_28022/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-31_20-48-56_712102_28022/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-07-31_20-48-56_712102_28022', 'metrics_export_port': 65424, 'gcs_address': '127.0.0.1:62015', 'address': '127.0.0.1:62015', 'node_id': '1cb8d7be96e2ee228185b556a30a7a11565d1e84d758ae15930eadc9'})

In [9]:
#옛날건 어떤가 확인해보기 레시피 크롤링
start = time()
recipe_path_list = ray.get(crawling_recipe_path_list.remote(13))
print(time()-start)

0.20711493492126465


In [12]:
start = time()
ref_list = []
for recipe_path in tqdm(recipe_path_list):
    ref_list.append(makeing_recipe.remote(recipe_path))
recipe_list = ray.get(ref_list)
c = (time()-start)
print(c)

100%|█████████████████████████████████████████| 40/40 [00:00<00:00, 3813.18it/s]


1.7472190856933594


In [15]:
recipe_list

[{'recipe_id': '6983822',
  'title': '수육국밥 한그릇 하시고 힘내세요 ♥',
  'size': '3인분',
  'ingredients': [{'name': '돼지고기', 'quantity': '2근'},
   {'name': '양파', 'quantity': '1.5개'},
   {'name': '통마늘', 'quantity': '20개'},
   {'name': '대파', 'quantity': '2-3대'},
   {'name': '소금', 'quantity': '1/2큰술'},
   {'name': '부추', 'quantity': '적당량'},
   {'name': '통후추', 'quantity': '1큰술'},
   {'name': '생강', 'quantity': '2-3쪽'},
   {'name': '물', 'quantity': '3.6L'},
   {'name': '다진마늘', 'quantity': '1큰술'},
   {'name': '소금', 'quantity': '1/3큰술'},
   {'name': '맛술', 'quantity': '2큰술'},
   {'name': '고춧가루', 'quantity': '2큰술'},
   {'name': '국간장', 'quantity': '1큰술'}],
  'sequences': [{'sequence': 1,
    'content': '고기는 칼집내서 끓는물에 3분정도 데쳐주세요. \n저는 칼집을 그 이후에 냈는데 미리 내주시는게 좋아요. 수육용은 좀 퉁퉁하니까요.'},
   {'sequence': 2,
    'content': '이제 데친 물은 다 버리고 다시 솥에 고기를 넣고 물 양을 고기의 3배정도로 넣어주세요. 저는 총 3.6L가 들어갔는데 양보다는 넉넉하게 넣어주세요!'},
   {'sequence': 3, 'content': '여기에 양파 1개 반'},
   {'sequence': 4, 'content': '통마늘 20개'},
   {'sequence': 5, 'conte

In [16]:
ray.shutdown()

In [13]:
ray.get(makeing_recipe.remote(recipe_path_list[0]))

RayTaskError(AttributeError): [36mray::makeing_recipe()[39m (pid=28098, ip=127.0.0.1)
  File "/var/folders/c7/t5bfdjyx00dcjsxfly9sv8hr0000gn/T/ipykernel_28022/2630134317.py", line 12, in makeing_recipe
AttributeError: 'NoneType' object has no attribute 'get'

In [14]:
makeing_recipe.remote(recipe_path_list[0])

ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)

2022-07-31 20:53:36,751	ERROR worker.py:92 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::makeing_recipe() (pid=28098, ip=127.0.0.1)
  File "/var/folders/c7/t5bfdjyx00dcjsxfly9sv8hr0000gn/T/ipykernel_28022/2630134317.py", line 12, in makeing_recipe
AttributeError: 'NoneType' object has no attribute 'get'


In [17]:
def makeing_recipe(recipe_path):
    url = basic_url+recipe_path
    response = requests.get(url, headers=headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        pass
    soup = BeautifulSoup(response.text, "html.parser")
    
    recipe = {}
    recipe['recipe_id'] = recipe_path.split("/")[2]
    recipe['img'] = soup.find("img", id="main_thumbs").get("src")
    try:
        recipe['title'] = soup.find("div","view2_summary st3").find("h3").text
    except:
        print(f"이름 오류 {recipe_path}")
        
    try:
        recipe['size'] = soup.find("span","view2_summary_info1").text
    except:
        print(f"몇인분 오류 {recipe_path}")
        
    try:
        recipe['ingredients'] = parsing_recipe_ingredients(soup)
    except:
        print(f"재료 오류 {recipe_path}")
        
    try:
        recipe['sequences'] = parsing_recipe_sequences(soup)
    except:
        print(f"조리법 오류 {recipe_path}")
    
    return recipe

makeing_recipe(recipe_path_list[0])

{'recipe_id': '6983984',
 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2022/07/21/da5947edcda79ca3603fabc973afa1ba1.jpg',
 'title': '쌈밥? 쌈밥!! 양배추와 함께 먹는 참치쌈장 레시피',
 'size': '1인분',
 'ingredients': [{'name': '참치캔', 'quantity': '2캔'},
  {'name': '양파', 'quantity': '1개'},
  {'name': '마늘', 'quantity': '3쪽'},
  {'name': '다진마늘', 'quantity': '1t'},
  {'name': '고추장', 'quantity': '1t'},
  {'name': '된장', 'quantity': '1t'},
  {'name': '기름', 'quantity': '약간'}],
 'sequences': [{'sequence': 1, 'content': '양파 1개와 마늘쪽을 썰어주세요'},
  {'sequence': 2, 'content': '기름을 두르고 썰어둔 양파와 마늘, 다진마늘 1스푼을 넣고 볶아줍니다'},
  {'sequence': 3, 'content': '양파, 마늘이 반정도 익었을 때 참치캔 2캔도 넣고 볶아주세요'},
  {'sequence': 4,
   'content': '고추장 1t, 된장1t를 넣고 볶아주세요\n설탕은 기호에 따라 추가해셔도 되지만 설탕을 넣지 않아도 충분히 맛있습니다'},
  {'sequence': 5, 'content': '된장, 고추장이 충분이 섞이면 끝입니다!'}]}